Outputs corpus, cleaned tweets with their createdAt timestamps
into a csv.

Currently does not stem or remove stop words (code is written
for it - the user just needs to set the appropriate booleans
to True at the top.

In [1]:
# !pip3 install nltk
import pandas as pd
import re
import html
import nltk
import string
# nltk.download() # run this the first time you run nltk
from nltk.corpus import stopwords
from nltk import PorterStemmer

In [2]:
# stop words and stemming
remove_stop = False # set this to True if want to only remove stop words
stem_remove_stop = False # set this to True if want to stem AND remove stop words

# add stop words
stop_words = stopwords.words('english')
alpha_lower_list = list(string.ascii_lowercase)
stop_words.extend(alpha_lower_list) # appends lowercase single letters of alphabet to stopwords.
stop_words.append('rt')
stop_words.append('nft')

In [5]:
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [2]:
# file paths
# TWEET_CORPUS_INPUT_FILE = "datain/clean/sample100k.jsonl"
# CLEANED_TWEETS_OUTPUT_FILE = "datain/topic_modelling/cleaned_tweets.csv"
TWEET_CORPUS_INPUT_FILE = "../../datain/clean/largest_community_tweets.jsonl"
CLEANED_TWEETS_OUTPUT_FILE = "../../datain/topic_modelling/cleaned_tweets_largest_community2.csv"
# BTM_CLEANED_TWEETS_OUTPUT_FILE = "datain/topic_modelling/cleaned_tweets_largest_community_btm_2.csv"
# BTM2_CLEANED_TWEETS_OUTPUT_FILE = "datain/topic_modelling/cleaned_tweets_largest_community_btm2_2.csv"

In [3]:
file_path = TWEET_CORPUS_INPUT_FILE
data = pd.read_json(file_path, lines=True)
data

,lang,reply_settings,public_metrics,text,possibly_sensitive,entities,created_at,author_id,conversation_id,referenced_tweets,id,context_annotations,source,in_reply_to_user_id,attachments,geo,withheld
0,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Good project \n#BSC @DinoSour #NFT\n@yamin_rah...,False,"{'hashtags': [{'start': 14, 'end': 18, 'tag': ...",2021-05-31 23:59:42+00:00,1397168952908779520,1399515966774530048,"[{'type': 'quoted', 'id': '1398339274953564163'}]",1399515966774530048,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter Web App,NaN,NaN,NaN,NaN
1,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@AromaFinance Great project!\n\n$reset #bsc #B...,False,"{'cashtags': [{'start': 30, 'end': 36, 'tag': ...",2021-05-31 23:59:40+00:00,1395244202808680448,1399313027065810944,"[{'type': 'replied_to', 'id': '139931302706581...",1399515957362450432,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter Web App,1.390241e+18,NaN,NaN,NaN
2,en,everyone,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",Such a beautiful project and congratulations t...,False,"{'hashtags': [{'start': 198, 'end': 213, 'tag'...",2021-05-31 23:59:35+00:00,1397350208489463808,1399515936093204480,"[{'type': 'quoted', 'id': '1399372631267287041'}]",1399515936093204480,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for Android,NaN,NaN,NaN,NaN
3,und,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@weyuofficial @thisisannsophie #NFT #NFTCommun...,False,"{'hashtags': [{'start': 31, 'end': 35, 'tag': ...",2021-05-31 23:59:24+00:00,1266110966271336448,1398992760380743680,"[{'type': 'replied_to', 'id': '139899276038074...",1399515888664006656,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter Web App,1.382288e+18,NaN,NaN,NaN
4,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",I am participating in the CryptoUltraman NFT a...,False,"{'hashtags': [{'start': 60, 'end': 78, 'tag': ...",2021-05-31 23:58:47+00:00,2718560166,1399515734007447552,"[{'type': 'quoted', 'id': '1398277372651081732'}]",1399515734007447552,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for Android,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459474,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@SMATOOS_now @bagasadys @airdrophunter78 @jher...,False,"{'mentions': [{'start': 0, 'end': 12, 'usernam...",2021-02-01 11:05:55+00:00,343817344,1349729014944972800,"[{'type': 'replied_to', 'id': '134972901494497...",1356197080272752640,NaN,Twitter for Android,3.339523e+08,NaN,NaN,NaN
459475,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@SMATOOS_now @bagasadys @airdrophunter78 @jher...,False,"{'mentions': [{'start': 0, 'end': 12, 'usernam...",2021-02-01 10:49:53+00:00,1099564686788374528,1349729014944972800,"[{'type': 'replied_to', 'id': '134972901494497...",1356193045817872384,NaN,Twitter for Android,3.339523e+08,NaN,NaN,NaN
459476,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@ZthCrypto @AlienWorlds How are NFT rates dete...,False,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",2021-02-01 09:29:19+00:00,1164164048046514176,1356157119909642240,"[{'type': 'replied_to', 'id': '135615711990964...",1356172769424244736,NaN,Twitter Web App,1.138926e+18,NaN,NaN,NaN
459477,und,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@JustLiquidity @JulSwap NFT?\n\n@NurmilaSinaga...,False,"{'mentions': [{'start': 0, 'end': 14, 'usernam...",2021-02-01 06:04:27+00:00,1078195838193561600,1356120833832480768,"[{'type': 'replied_to', 'id': '135612083383248...",1356121214108876800,NaN,Twitter for Android,1.288886e+18,NaN,NaN,NaN


In [8]:
def load_data():
    '''
    Import corpus data in json format.
    Filter to have only english tweets and remove retweets.

    @return imported english, non-retweeted data
    '''
    #import the data
    file_path = TWEET_CORPUS_INPUT_FILE
    data = pd.read_json(file_path, lines=True)

    # get first 100k lines of a larger dataset
    # data = data.iloc[0:100000] # select first 100k entries of input file

    # clean data: remove retweets and select only english tweets
    data = data[~data["text"].apply(lambda x: x.startswith("RT"))]
    data = data[data["lang"].apply(lambda x: x == "en")]
    data = data.rename(columns={'text': 'corpus'})

    return data


def clean_tweet(tweet):
    '''
    Cleans tweet from hashtags, mentions, special characters, html entities, numbers,
    links, and (optionally) stop words & stemming. Converts text to lower case.

    @param tweet - a single tweet (String)
    @return cleaned tweet (String)
    '''
    tweet = str.lower(tweet)
    tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)|(#[A-Za-z0-9_]+)", " ", tweet).split()) # remove mentions and hashtags
    tweet = re.sub("(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)", "", tweet, flags=re.MULTILINE) # remove links
    tweet = re.sub("0x([\da-z\.-]+)", "", tweet, flags=re.MULTILINE) # remove addresses/pointers
    tweet = re.sub('\&\w+', "", tweet) # remove html entities (example &amp)
    tweet = re.sub('[^a-zA-Z#,.?!\-\'();: ]+', ' ', tweet) # make sure tweet is only letters

    # TODO - decide whether want to stem and remove stop words (default set to False)
    if remove_stop:
        tweet = remove_stopwords(tweet)
    elif stem_remove_stop:
        tweet = stem_and_remove_stopwords()
    else:
        # convert cleaned tweet to list (each tweet being one element in the list)
        tweet = ' '.join([word for word in tweet.split()])

    return tweet


def remove_stopwords(tweet):
    '''
    Remove stop words from the given tweet.

    @param tweet - a single (cleaned) tweet (String)
    @return tweet without stopwords (String)
    '''
    return' '.join([word for word in tweet.split() if word not in stop_words])

# def stem_and_remove_stopwords(tweet):
#     '''
#     Remove stop words from the given tweet and stem the words in the tweet.

#     @param tweet - a single (cleaned) tweet (String)
#     @return stemmed tweet without stopwords (String)
#     '''
#     # TODO: perhaps use Vader's stemming function instead of PorterStemmer
#     return ' '.join([PorterStemmer().stem(word=word) for word in tweet.split()

In [6]:
'''
Main running code that executes all cleaning corpus functions in the
correct order for the pipeline.
'''
print("Cleaning corpus...")
df = load_data()
df

Cleaning corpus...


,lang,reply_settings,public_metrics,corpus,possibly_sensitive,entities,created_at,author_id,conversation_id,referenced_tweets,id,context_annotations,source,in_reply_to_user_id,attachments,geo,withheld
0,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Good project \n#BSC @DinoSour #NFT\n@yamin_rah...,False,"{'hashtags': [{'start': 14, 'end': 18, 'tag': ...",2021-05-31 23:59:42+00:00,1397168952908779520,1399515966774530048,"[{'type': 'quoted', 'id': '1398339274953564163'}]",1399515966774530048,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter Web App,NaN,NaN,NaN,NaN
1,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@AromaFinance Great project!\n\n$reset #bsc #B...,False,"{'cashtags': [{'start': 30, 'end': 36, 'tag': ...",2021-05-31 23:59:40+00:00,1395244202808680448,1399313027065810944,"[{'type': 'replied_to', 'id': '139931302706581...",1399515957362450432,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter Web App,1.390241e+18,NaN,NaN,NaN
2,en,everyone,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",Such a beautiful project and congratulations t...,False,"{'hashtags': [{'start': 198, 'end': 213, 'tag'...",2021-05-31 23:59:35+00:00,1397350208489463808,1399515936093204480,"[{'type': 'quoted', 'id': '1399372631267287041'}]",1399515936093204480,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for Android,NaN,NaN,NaN,NaN
4,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",I am participating in the CryptoUltraman NFT a...,False,"{'hashtags': [{'start': 60, 'end': 78, 'tag': ...",2021-05-31 23:58:47+00:00,2718560166,1399515734007447552,"[{'type': 'quoted', 'id': '1398277372651081732'}]",1399515734007447552,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for Android,NaN,NaN,NaN,NaN
5,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@apenftorg @CoinMarketCap Nice to find this pr...,False,"{'cashtags': [{'start': 216, 'end': 220, 'tag'...",2021-05-31 23:58:44+00:00,1383794353760391168,1397848170739077120,"[{'type': 'replied_to', 'id': '139784817073907...",1399515723274280960,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter Web App,1.392094e+18,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459473,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",$DENA #NFT #Defi #YieldFarming\nGreat \n@lensa...,False,"{'hashtags': [{'start': 6, 'end': 10, 'tag': '...",2021-02-01 11:31:45+00:00,1322618452108931072,1356203583193063424,"[{'type': 'quoted', 'id': '1349729014944972800'}]",1356203583193063424,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for Android,NaN,NaN,NaN,NaN
459474,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@SMATOOS_now @bagasadys @airdrophunter78 @jher...,False,"{'mentions': [{'start': 0, 'end': 12, 'usernam...",2021-02-01 11:05:55+00:00,343817344,1349729014944972800,"[{'type': 'replied_to', 'id': '134972901494497...",1356197080272752640,NaN,Twitter for Android,3.339523e+08,NaN,NaN,NaN
459475,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@SMATOOS_now @bagasadys @airdrophunter78 @jher...,False,"{'mentions': [{'start': 0, 'end': 12, 'usernam...",2021-02-01 10:49:53+00:00,1099564686788374528,1349729014944972800,"[{'type': 'replied_to', 'id': '134972901494497...",1356193045817872384,NaN,Twitter for Android,3.339523e+08,NaN,NaN,NaN
459476,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@ZthCrypto @AlienWorlds How are NFT rates dete...,False,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",2021-02-01 09:29:19+00:00,1164164048046514176,1356157119909642240,"[{'type': 'replied_to', 'id': '135615711990964...",1356172769424244736,NaN,Twitter Web App,1.138926e+18,NaN,NaN,NaN


In [9]:
# clean data text line by line
# cleaned_text = []
# for index in df.index:
#     text = df["corpus"][index]
#     cleaned_text.append(clean_tweet(text))

df['cleaned_tweet'] = df['corpus'].apply(clean_tweet)
df
# df['cleaned_text'] = cleaned_text

,lang,reply_settings,public_metrics,corpus,possibly_sensitive,entities,created_at,author_id,conversation_id,referenced_tweets,id,context_annotations,source,in_reply_to_user_id,attachments,geo,withheld,cleaned_tweet
0,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Good project \n#BSC @DinoSour #NFT\n@yamin_rah...,False,"{'hashtags': [{'start': 14, 'end': 18, 'tag': ...",2021-05-31 23:59:42+00:00,1397168952908779520,1399515966774530048,"[{'type': 'quoted', 'id': '1398339274953564163'}]",1399515966774530048,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter Web App,NaN,NaN,NaN,NaN,good project
1,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@AromaFinance Great project!\n\n$reset #bsc #B...,False,"{'cashtags': [{'start': 30, 'end': 36, 'tag': ...",2021-05-31 23:59:40+00:00,1395244202808680448,1399313027065810944,"[{'type': 'replied_to', 'id': '139931302706581...",1399515957362450432,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter Web App,1.390241e+18,NaN,NaN,NaN,great project! reset
2,en,everyone,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",Such a beautiful project and congratulations t...,False,"{'hashtags': [{'start': 198, 'end': 213, 'tag'...",2021-05-31 23:59:35+00:00,1397350208489463808,1399515936093204480,"[{'type': 'quoted', 'id': '1399372631267287041'}]",1399515936093204480,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for Android,NaN,NaN,NaN,NaN,such a beautiful project and congratulations t...
4,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",I am participating in the CryptoUltraman NFT a...,False,"{'hashtags': [{'start': 60, 'end': 78, 'tag': ...",2021-05-31 23:58:47+00:00,2718560166,1399515734007447552,"[{'type': 'quoted', 'id': '1398277372651081732'}]",1399515734007447552,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for Android,NaN,NaN,NaN,NaN,i am participating in the cryptoultraman nft a...
5,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@apenftorg @CoinMarketCap Nice to find this pr...,False,"{'cashtags': [{'start': 216, 'end': 220, 'tag'...",2021-05-31 23:58:44+00:00,1383794353760391168,1397848170739077120,"[{'type': 'replied_to', 'id': '139784817073907...",1399515723274280960,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter Web App,1.392094e+18,NaN,NaN,NaN,"nice to find this project, this project will b..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459473,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",$DENA #NFT #Defi #YieldFarming\nGreat \n@lensa...,False,"{'hashtags': [{'start': 6, 'end': 10, 'tag': '...",2021-02-01 11:31:45+00:00,1322618452108931072,1356203583193063424,"[{'type': 'quoted', 'id': '1349729014944972800'}]",1356203583193063424,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for Android,NaN,NaN,NaN,NaN,dena great
459474,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@SMATOOS_now @bagasadys @airdrophunter78 @jher...,False,"{'mentions': [{'start': 0, 'end': 12, 'usernam...",2021-02-01 11:05:55+00:00,343817344,1349729014944972800,"[{'type': 'replied_to', 'id': '134972901494497...",1356197080272752640,NaN,Twitter for Android,3.339523e+08,NaN,NaN,NaN,join gays dena
459475,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@SMATOOS_now @bagasadys @airdrophunter78 @jher...,False,"{'mentions': [{'start': 0, 'end': 12, 'usernam...",2021-02-01 10:49:53+00:00,1099564686788374528,1349729014944972800,"[{'type': 'replied_to', 'id': '134972901494497...",1356193045817872384,NaN,Twitter for Android,3.339523e+08,NaN,NaN,NaN,a great project
459476,en,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@ZthCrypto @AlienWorlds How are NFT rates dete...,False,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",2021-02-01 09:29:19+00:00,1164164048046514176,1356157119909642240,"[{'type': 'replied_to', 'id': '135615711990964...",1356172769424244736,NaN,Twitter Web App,1.138926e+18,NaN,NaN

In [ ]:
# cleaned_text

In [15]:
cleaned_df = df.loc[:, ["id", "corpus", "cleaned_tweet"]]
cleaned_df

,id,corpus,cleaned_tweet
0,1399515966774530048,Good project \n#BSC @DinoSour #NFT\n@yamin_rah...,good project
1,1399515957362450432,@AromaFinance Great project!\n\n$reset #bsc #B...,great project! reset
2,1399515936093204480,Such a beautiful project and congratulations t...,such a beautiful project and congratulations t...
4,1399515734007447552,I am participating in the CryptoUltraman NFT a...,i am participating in the cryptoultraman nft a...
5,1399515723274280960,@apenftorg @CoinMarketCap Nice to find this pr...,"nice to find this project, this project will b..."
...,...,...,...
459473,1356203583193063424,$DENA #NFT #Defi #YieldFarming\nGreat \n@lensa...,dena great
459474,1356197080272752640,@SMATOOS_now @bagasadys @airdrophunter78 @jher...,join gays dena
459475,1356193045817872384,@SMATOOS_now @bagasadys @airdrophunter78 @jher...,a great project
459476,1356172769424244736,@ZthCrypto @AlienWorlds How are NFT rates dete...,how are nft rates determined? higher nft chanc...


In [19]:
# cleaned_df = pd.DataFrame(cleaned_text[0:], columns=["cleaned_tweet"])
# cleaned_df

In [ ]:
# df['cleaned_tweet'] = cleaned_df['cleaned_tweet']
# df

In [20]:
# output data text, cleaned_tweets, and createdAt to csv
selected_columns = ["created_at", "id", "cleaned_tweet"]
df.to_csv(CLEANED_TWEETS_OUTPUT_FILE, columns = selected_columns)

# selected_columns = ["cleaned_tweet"]
# df.to_csv(BTM_CLEANED_TWEETS_OUTPUT_FILE, columns = selected_columns)
#
# selected_columns = ["id", "cleaned_tweet"]
# df.to_csv(BTM2_CLEANED_TWEETS_OUTPUT_FILE, columns = selected_columns, index=None)

print("Finished cleaning corpus...")

Finished cleaning corpus...
